In [2]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def transform_position_orientation(box_pos, box_quat, frame_pos, frame_quat):
    """
    Transform the position and orientation of an object from one coordinate frame to another.

    Parameters:
    - box_pos: Tuple of (x, y, z) coordinates of the box in the global frame.
    - box_quat: Tuple of (x, y, z, w) quaternion representing the box's orientation in the global frame.
    - frame_pos: Tuple of (x, y, z) coordinates of the new frame's origin in the global frame.
    - frame_quat: Tuple of (x, y, z, w) quaternion representing the new frame's orientation relative to the global frame.

    Returns:
    - Tuple of (new_pos, new_quat) where:
      - new_pos is the transformed position of the box in the new frame's coordinates.
      - new_quat is the transformed orientation of the box in the new frame's coordinates.
    """
    
    # Convert the input tuples to numpy arrays and quaternions
    box_pos = np.array(box_pos)
    frame_pos = np.array(frame_pos)
    box_rotation = R.from_quat(box_quat)
    frame_rotation = R.from_quat(frame_quat)

    # Compute the relative rotation from the frame to the global frame
    frame_rotation_inv = frame_rotation.inv()

    # Calculate the new position: translate box position to frame's coordinate system
    new_pos = frame_rotation_inv.apply(box_pos - frame_pos)

    # Calculate the new orientation: rotate box's quaternion by the inverse of the frame's quaternion
    new_quat = (frame_rotation_inv * box_rotation).as_quat()

    return (new_pos, new_quat)

def quaternion_to_euler(quaternion):
    rotation = R.from_quat(quaternion)
    euler_angles = rotation.as_euler('xyz', degrees=True)  # or 'zyx' based on your convention
    return euler_angles

# Example usage
box_position = (0.12853120767649265, 0.4999952337139436, 0.07498561387601738)  # Box's position in the global frame
box_orientation = (7.957936148724259e-10, 5.75503262204572e-05, -1.210851474082113e-06, 0.9999999983432468)  # Box's orientation as quaternion
frame_position = (5, 0, 0)  # Second frame's position in the global frame
frame_orientation = (0, 0, 0, 1)  # Second frame's orientation (no rotation)

new_position, new_orientation_quat = transform_position_orientation(
    box_position, box_orientation, frame_position, frame_orientation)

new_orientation_euler = quaternion_to_euler(new_orientation_quat)

print("New Position:", new_position)
print("New Orientation (Euler angles, degrees):", new_orientation_euler)


New Position: [-4.87146879  0.49999523  0.07498561]
New Orientation (Euler angles, degrees): [ 8.32059304e-08  6.59478161e-03 -1.38753354e-04]


In [10]:
#!/usr/bin/env python
import rospy
from trac_ik_python.srv import GetPositionIK
from moveit_msgs.msg import PositionIKRequest

def request_ik(pose):
    rospy.wait_for_service('compute_ik')
    try:
        compute_ik = rospy.ServiceProxy('compute_ik', GetPositionIK)
        ik_request = PositionIKRequest()
        ik_request.pose_stamped = pose  # Your desired pose
        response = compute_ik(ik_request)
        return response.solution.joint_state
    except rospy.ServiceException as e:
        print("Service call failed: %s"%e)

def main():
    rospy.init_node('ik_client')
    # Example pose setup
    # pose = setup your PoseStamped message here
    result = request_ik(pose)
    print("IK Result:", result)

if __name__ == '__main__':
    main()


ModuleNotFoundError: No module named 'trac_ik_python'